In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from output import *
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout,Input,Concatenate
from sklearn import datasets, linear_model
from sklearn.model_selection import KFold,cross_val_score
from sklearn import preprocessing
from keras.wrappers.scikit_learn import KerasRegressor
from keras import regularizers
from sklearn.preprocessing import PolynomialFeatures
from pandas.tools.plotting import scatter_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from keras.utils.np_utils import to_categorical

In [8]:
def create_model_functional():
    coupling_input = Input(shape=(8,), dtype='float32', name='coupling_input')
    x = Dense(64, activation='relu',activity_regularizer=regularizers.l2(0.01))(coupling_input)
    x = Dense(32,activation='relu')(x)
    x = Dense(16,activation='relu',activity_regularizer=regularizers.l2(0.01))(x)
    cohesion_input = Input(shape=(2,), dtype='float32', name='cohesion_input')
    x = keras.layers.concatenate([x, cohesion_input])
    x_ = Dense(8,activation='relu',activity_regularizer=regularizers.l2(0.01))(x)
    main_output=Dense(1,activation='sigmoid',name='output')(x)
    badSmell_output=Dense(4,activation='softmax',name='badSmell')(x)
    model = Model([coupling_input, cohesion_input], [main_output,badSmell_output])
    model.compile(optimizer='sgd', loss='mean_squared_error',metrics=['acc'])
    model.summary()
    return model

In [9]:
def load_data():
    x,y,z,y_badSmells=generate()
    junit_in=x[:z[0]]
    junit_out=y[:z[0]]
    ganttproject_in=x[:z[1]]
    ganttproject_out=y[:z[1]]
    mockito_in=x[:z[2]]
    mockito_out=y[:z[2]]
    x_train_real=preprocessing.scale(np.array(x))
    x_test_real=preprocessing.scale(np.array(ganttproject_in))
    y_train_real=np.array(y)
    y_test_real=np.array(ganttproject_out)
    y_bs = to_categorical(y_badSmells)		  
    return x_train_real,y_train_real,y_bs

In [10]:
def train_and_evaluate__model(model, data_train, labels_train,labels2, data_test, labels_test,labels2_test):
    dt1=data_train[:,:8]
    dt2=data_train[:,8:]
    history=model.fit([dt1, dt2],[labels_train,labels2],nb_epoch=200, batch_size=10,verbose=0)
    print(model.metrics_names)
    print(model.evaluate([data_test[:,:8],data_test[:,8:]], [labels_test,labels2_test],batch_size=10))


In [11]:
data, labels, badSmells = load_data()
model = None # Clearing the NN.
model = create_model_functional()
train_and_evaluate__model(model, data[:1500], labels[:1500], badSmells[:1500],data[1500:], labels[1500:],badSmells[1500:])


/home/ran/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype <U32 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/ran/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype <U32 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
W0718 21:59:38.149072 140390829819648 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0718 21:59:38.168644 140390829819648 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0718 21:59:38.170380 140390829819648 deprecation_wrapper.py:119] From /usr/l

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
coupling_input (InputLayer)     (None, 8)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           576         coupling_input[0][0]             
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 32)           2080        dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 16)           528         dense_2[0][0]                    
__________________________________________________________________________________________________
cohesion_i

W0718 21:59:38.556254 140390829819648 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0718 21:59:38.607340 140390829819648 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



['loss', 'output_loss', 'badSmell_loss', 'output_acc', 'badSmell_acc']
619/619 [==============================] - 0s 118us/step
[0.21300998096608575, 0.09184869636310586, 0.09250793390589886, 0.8820678454030889, 0.7883683317890074]
